In [24]:
#导入库并产生数据
import math
import numpy as np
import torch
from torch.utils import data
import random

#定义正态分布
def normal(x,mu,sigma):
    p=1/math.sqrt(2*math.pi*sigma*sigma)
    return p*np.exp(-0.5/sigma**2 *(x-mu)**2)

#制造数据集 y=Xw+b+噪声
def synthetic_data(w, b, num_examples): 
    X = torch.normal(0, 1, (num_examples, len(w)))
    y = torch.matmul(X, w) + b
    y += torch.normal(0, 0.01, y.shape)
    return X, y.reshape((-1, 1))

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 100)

In [25]:
#"""构造⼀个PyTorch数据迭代器"""
def load_array(data_arrays, batch_size, is_train=True): 
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)
batch_size = 10
data_iter = load_array((features, labels), batch_size)
print(next(iter(data_iter)))

[tensor([[-0.0973,  1.7949],
        [-0.1674,  1.3817],
        [-0.3620,  0.7463],
        [-0.8949,  0.0140],
        [ 2.2519, -1.2528],
        [ 0.6971,  0.6998],
        [ 1.1333, -1.9965],
        [ 0.7712, -0.3163],
        [-0.8713, -0.7706],
        [-0.6275,  1.0607]]), tensor([[-2.1129],
        [-0.8391],
        [ 0.9528],
        [ 2.3703],
        [12.9722],
        [ 3.2046],
        [13.2660],
        [ 6.8286],
        [ 5.0957],
        [-0.6548]])]


In [26]:
#建立模型
from torch import nn
#nn.linear第一个参数为输入的特征，第二个为输出量
net=nn.Sequential(nn.Linear(2,1))
#初始化模型参数
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)
#定义损失函数
loss = nn.MSELoss()
#定义优化器
sgd=torch.optim.SGD(net.parameters(), lr=0.03)


In [27]:
num_epochs = 10
for epoch in range(num_epochs):
    for X,y in data_iter:
        l=loss(net(X),y)
        sgd.zero_grad()
        l.backward()
        sgd.step()
    l=loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {float(l):f}')



epoch 1, loss 9.709099
epoch 2, loss 3.258214
epoch 3, loss 1.118123
epoch 4, loss 0.389076
epoch 5, loss 0.136190
epoch 6, loss 0.048256
epoch 7, loss 0.017291
epoch 8, loss 0.006289
epoch 9, loss 0.002324
epoch 10, loss 0.000902
